In [1]:

from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# re-size all the images
IMAGE_SIZE = [224, 224]
train_path = '/content/drive/MyDrive/chest_xray/train'
valid_path = '/content/drive/MyDrive/chest_xray/test'


In [4]:
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

94765736/94765736 [==============================] - 0s 0us/step


Making all the layers non traininable and droping out 1st layer so that we can use our own image size

In [5]:
for layer in resnet.layers:
    layer.trainable = False

In [6]:
folders = glob('/content/drive/MyDrive/chest_xray/train/*')

In [7]:
x = Flatten()(resnet.output)

In [8]:
prediction = Dense(len(folders), activation='softmax')(x)
 
# creating a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [10]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

Data augmentation

In [11]:


 
# Use the Image Data Generator to import the images from the dataset
from keras.preprocessing.image import ImageDataGenerator
 
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
 
test_datagen = ImageDataGenerator(rescale = 1./255)


########################################################Generating the batches of augmentated data#############################################################


# providing the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
 

 
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')


Found 5224 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


Training and evaluating the model

In [12]:
# fiting the model
 
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=8, #10,15,20
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)


Epoch 1/8
164/164 [==============================] - 1199s 7s/step - loss: 0.5969 - accuracy: 0.7959 - val_loss: 0.3956 - val_accuracy: 0.8157
Epoch 2/8
164/164 [==============================] - 532s 3s/step - loss: 0.3215 - accuracy: 0.8656 - val_loss: 0.4725 - val_accuracy: 0.7997
Epoch 3/8
164/164 [==============================] - 529s 3s/step - loss: 0.3519 - accuracy: 0.8645 - val_loss: 1.5242 - val_accuracy: 0.6394
Epoch 4/8
164/164 [==============================] - 536s 3s/step - loss: 0.3680 - accuracy: 0.8660 - val_loss: 0.3749 - val_accuracy: 0.8365
Epoch 5/8
164/164 [==============================] - 537s 3s/step - loss: 0.3383 - accuracy: 0.8771 - val_loss: 0.3630 - val_accuracy: 0.8494
Epoch 6/8
164/164 [==============================] - 534s 3s/step - loss: 0.2942 - accuracy: 0.8928 - val_loss: 0.6919 - val_accuracy: 0.7628
Epoch 7/8
164/164 [==============================] - 528s 3s/step - loss: 0.2486 - accuracy: 0.9028 - val_loss: 0.3416 - val_accuracy: 0.8750
Epoch

In [13]:
import tensorflow as tf
from keras.models import load_model
model.save('resnet50pneumonia.h5')